In [198]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt

In [199]:
### reading the datset

data=pd.read_csv("train.csv")

In [200]:
#First we need to deal with the missing values
#There are 2 columns with missing values LotFrontage and GarageYrBlt
#LotFrontage is a numerical column and hence we impute it with the median 
#We feel the GarageYrBlt is not a useful column in 

In [201]:
data.describe()

,s,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [202]:
data['MasVnrType'].isnull().sum()

8

In [203]:
data['MasVnrType'].unique()

array(['BrkFace', 'None', 'Stone', 'BrkCmn', nan], dtype=object)

In [204]:
#At first we split the train data into training and validation inorder to test how the model works

In [205]:
len(data)

1460

In [206]:
data_1=shuffle(data)

In [207]:
data_1

,s,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
539,540,20,RL,NaN,11423,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,2000,5,2010,WD,Normal,272000
791,792,80,RL,NaN,11333,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,146800
816,817,20,RL,NaN,11425,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,137000
989,990,60,FV,65.0,8125,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2006,New,Partial,197000
890,891,50,RL,60.0,8064,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,2000,7,2007,WD,Normal,122900
1291,1292,160,RM,21.0,1680,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2009,WD,Normal,119500
272,273,60,RL,92.0,11764,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,290000
794,795,60,RL,NaN,10832,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2008,WD,Normal,194500
1019,1020,120,RL,43.0,3013,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,213490
515,516,20,RL,94.0,12220,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2009,New,Partial,402861


In [208]:
train_data=data_1[:int(len(data_1)*0.60)]

In [209]:
len(train_data)

876

In [210]:
train_data

,s,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
539,540,20,RL,NaN,11423,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,2000,5,2010,WD,Normal,272000
791,792,80,RL,NaN,11333,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,146800
816,817,20,RL,NaN,11425,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,137000
989,990,60,FV,65.0,8125,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2006,New,Partial,197000
890,891,50,RL,60.0,8064,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,2000,7,2007,WD,Normal,122900
1291,1292,160,RM,21.0,1680,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2009,WD,Normal,119500
272,273,60,RL,92.0,11764,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,290000
794,795,60,RL,NaN,10832,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2008,WD,Normal,194500
1019,1020,120,RL,43.0,3013,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,213490
515,516,20,RL,94.0,12220,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2009,New,Partial,402861


In [211]:
val_data=data_1[int(len(data_1)*0.60):]

In [212]:
len(val_data)

584

In [213]:
val_data

,s,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1051,1052,20,RL,103.0,11175,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2007,New,Partial,200141
131,132,60,RL,NaN,12224,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,244000
599,600,160,RM,24.0,1950,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,7,2008,COD,Normal,151000
1451,1452,20,RL,78.0,9262,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,New,Partial,287090
427,428,20,RL,77.0,8593,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,109008
1431,1432,120,RL,NaN,4928,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2009,WD,Normal,143750
1307,1308,20,RL,60.0,8072,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,138000
1082,1083,20,RL,70.0,8749,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2007,WD,Normal,192000
930,931,20,RL,73.0,8925,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,201000
857,858,60,RL,65.0,8125,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2009,WD,Normal,174000


In [214]:
len(train_data)+len(val_data)

1460

In [215]:
# now we need to impute the missing values in test and training datasets
# there are 2 columns with missing values "Lot Frontage" and "GarageYrBlt"

In [216]:
#Imputing the training data set columns

In [217]:
imp=Imputer(missing_values="NaN", strategy="median",axis=1 ) #specify axis
q = imp.fit_transform(train_data["LotFrontage"]).T #perform a transpose operation
train_data['LotFrontage']=q
print (train_data)

C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


         s  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
539    540          20       RL         68.0    11423   Pave   NaN      Reg   
791    792          80       RL         68.0    11333   Pave   NaN      IR1   
816    817          20       RL         68.0    11425   Pave   NaN      IR1   
989    990          60       FV         65.0     8125   Pave   NaN      Reg   
890    891          50       RL         60.0     8064   Pave   NaN      Reg   
1291  1292         160       RM         21.0     1680   Pave   NaN      Reg   
272    273          60       RL         92.0    11764   Pave   NaN      IR1   
794    795          60       RL         68.0    10832   Pave   NaN      IR1   
1019  1020         120       RL         43.0     3013   Pave   NaN      Reg   
515    516          20       RL         94.0    12220   Pave   NaN      Reg   
273    274          20       RL         80.0     9600   Pave   NaN      Reg   
494    495          30       RM         50.0     578

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [218]:
train_data['LotFrontage'].median()

68.0

In [219]:
imp=Imputer(missing_values="NaN",strategy="most_frequent",axis=1)
q=imp.fit_transform(train_data['GarageYrBlt']).T
train_data['GarageYrBlt']=q
print (train_data['GarageYrBlt'])

539     2001.0
791     1976.0
816     1954.0
989     2006.0
890     2003.0
1291    1972.0
272     1999.0
794     1994.0
1019    2005.0
515     2009.0
273     1958.0
494     1938.0
1168    1935.0
665     2000.0
278     2007.0
198     2005.0
645     1979.0
713     1970.0
187     1916.0
941     1999.0
1171    1958.0
598     1977.0
748     1996.0
865     1973.0
1287    1964.0
1158    2008.0
1192    1925.0
872     1953.0
1011    2005.0
392     1959.0
         ...  
576     1928.0
1030    2005.0
823     1940.0
1235    1914.0
822     2003.0
578     2007.0
279     1977.0
234     2002.0
154     1923.0
1165    2009.0
1386    1978.0
836     1948.0
867     1961.0
345     1939.0
1000    1956.0
753     2005.0
80      1968.0
975     2000.0
404     1995.0
466     1970.0
362     2003.0
331     1958.0
786     1961.0
1255    1978.0
1170    1977.0
1254    2003.0
386     2005.0
274     1982.0
784     2005.0
496     1992.0
Name: GarageYrBlt, dtype: float64


C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [220]:
#Imputing the test data set columns

In [221]:
imp=Imputer(missing_values="NaN", strategy="median",axis=1 ) #specify axis
q = imp.fit_transform(val_data["LotFrontage"]).T #perform a transpose operation
val_data['LotFrontage']=q
print (val_data['LotFrontage'])

1051    103.0
131      70.0
599      24.0
1451     78.0
427      77.0
1431     70.0
1307     60.0
1082     70.0
930      73.0
857      65.0
1455     62.0
1096     60.0
1071     78.0
434      21.0
942      42.0
336      86.0
1178     54.0
1434     80.0
233      75.0
376      57.0
92       80.0
1353     56.0
672      70.0
313     150.0
1305    108.0
493      70.0
1447     80.0
467      79.0
267      60.0
409      85.0
        ...  
504      24.0
692      42.0
140      70.0
1308    100.0
517      79.0
499      70.0
1217     72.0
1390     70.0
320     111.0
63       50.0
304      87.0
559      70.0
1348     70.0
422     100.0
622      71.0
301      66.0
1440     79.0
818      80.0
1401     62.0
203      70.0
508      60.0
524      95.0
1362     70.0
461      60.0
698      65.0
789      70.0
86      122.0
639      53.0
48       33.0
97       73.0
Name: LotFrontage, dtype: float64


C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [222]:
imp=Imputer(missing_values="NaN",strategy="most_frequent",axis=1)
q=imp.fit_transform(val_data['GarageYrBlt']).T
val_data['GarageYrBlt']=q
print (val_data['GarageYrBlt'])

1051    2007.0
131     2000.0
599     1980.0
1451    2008.0
427     1964.0
1431    1976.0
1307    2000.0
1082    2002.0
930     2007.0
857     1994.0
1455    1999.0
1096    2006.0
1071    1968.0
434     2006.0
942     2006.0
336     2005.0
1178    1921.0
1434    1977.0
233     1976.0
376     1998.0
92      1921.0
1353    1996.0
672     1977.0
313     1965.0
1305    2006.0
493     1964.0
1447    1995.0
467     1942.0
267     1939.0
409     2008.0
         ...  
504     1974.0
692     1989.0
140     2006.0
1308    1948.0
517     1996.0
499     1958.0
1217    2009.0
1390    2000.0
320     2006.0
63      1990.0
304     2003.0
559     2003.0
1348    1998.0
422     1954.0
622     1986.0
301     1998.0
1440    1993.0
818     1971.0
1401    2004.0
203     2004.0
508     1928.0
524     1996.0
1362    1920.0
461     1971.0
698     1973.0
789     1966.0
86      2005.0
639     2006.0
48      2006.0
97      1965.0
Name: GarageYrBlt, dtype: float64


C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [223]:
#There are many columns in the data which have "None" as a value which actually has a meaning.

In [224]:
a=pd.DataFrame(data.isnull().sum())
a['count']=pd.DataFrame(data.isnull().sum())

In [225]:
a[a['count']>0]

,0,count
LotFrontage,259,259
Alley,1369,1369
MasVnrType,8,8
MasVnrArea,8,8
BsmtQual,37,37
BsmtCond,37,37
BsmtExposure,38,38
BsmtFinType1,37,37
BsmtFinType2,38,38
Electrical,1,1


In [226]:
len(a[a['count']>0])

19

In [227]:
#Alley Column

In [228]:
train_data['Alley']=train_data['Alley'].replace(np.NaN, 'No Alley Access')

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [229]:
val_data['Alley']=val_data['Alley'].replace(np.NaN, 'No Alley Access')

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [230]:
train_data['Alley'].head()

539    No Alley Access
791    No Alley Access
816    No Alley Access
989    No Alley Access
890    No Alley Access
Name: Alley, dtype: object

In [231]:
val_data['Alley'].head()

1051    No Alley Access
131     No Alley Access
599     No Alley Access
1451    No Alley Access
427     No Alley Access
Name: Alley, dtype: object

In [232]:
#MasVnrType Column

In [233]:
train_data['MasVnrType'].unique()

array(['BrkFace', 'None', 'BrkCmn', 'Stone', nan], dtype=object)

In [234]:
train_data['MasVnrType']=train_data['MasVnrType'].replace(np.NaN,train_data['MasVnrType'].mode().values[0])

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [235]:
train_data['MasVnrType'].isnull().sum()

0

In [236]:
val_data['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', nan, 'BrkCmn'], dtype=object)

In [237]:
val_data['MasVnrType'].mode().values[0]

'None'

In [238]:
val_data['MasVnrType']=val_data['MasVnrType'].replace(np.NaN,val_data['MasVnrType'].mode().values[0])

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [239]:
val_data['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn'], dtype=object)

In [240]:
#MasVnrArea Column

In [241]:
train_data['MasVnrArea'].isnull().sum()

4

In [242]:
train_data['MasVnrArea'].median()

0.0

In [243]:
train_data['MasVnrArea']=train_data['MasVnrArea'].replace(np.NaN,train_data['MasVnrArea'].median())

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [244]:
val_data['MasVnrArea'].isnull().sum()

4

In [245]:
val_data['MasVnrArea'].median()

0.0

In [246]:
val_data['MasVnrArea']=val_data['MasVnrArea'].replace(np.NaN,val_data['MasVnrArea'].median())

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [247]:
#BsmtQual Column

In [248]:
train_data['BsmtQual'].isnull().sum()

24

In [249]:
train_data['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa', nan], dtype=object)

In [250]:
train_data['BsmtQual']=train_data['BsmtQual'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [251]:
val_data['BsmtQual'].isnull().sum()

13

In [252]:
val_data['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', nan, 'Fa'], dtype=object)

In [253]:
val_data['BsmtQual']=val_data['BsmtQual'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [254]:
#BsmtCond Column

In [255]:
train_data['BsmtCond'].isnull().sum()

24

In [256]:
train_data['BsmtCond'].unique()

array(['TA', 'Gd', nan, 'Fa', 'Po'], dtype=object)

In [257]:
train_data['BsmtCond']=train_data['BsmtCond'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [258]:
val_data['BsmtCond'].isnull().sum()

13

In [259]:
val_data['BsmtCond'].unique()

array(['TA', 'Gd', nan, 'Fa'], dtype=object)

In [260]:
val_data['BsmtCond']=val_data['BsmtCond'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [261]:
#BsmtExposure column

In [262]:
train_data['BsmtExposure'].isnull().sum()

25

In [263]:
train_data['BsmtExposure'].unique()

array(['Av', 'No', 'Mn', 'Gd', nan], dtype=object)

In [264]:
train_data['BsmtExposure']=train_data['BsmtExposure'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [265]:
val_data['BsmtExposure'].isnull().sum()

13

In [266]:
val_data['BsmtExposure'].unique()

array(['Av', 'No', 'Gd', 'Mn', nan], dtype=object)

In [267]:
val_data['BsmtExposure']=val_data['BsmtExposure'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [268]:
#BsmtFinType1 Column

In [269]:
train_data['BsmtFinType1'].isnull().sum()

24

In [270]:
train_data['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'BLQ', 'Unf', 'Rec', nan, 'LwQ'], dtype=object)

In [271]:
train_data['BsmtFinType1']=train_data['BsmtFinType1'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [272]:
val_data['BsmtFinType1'].isnull().sum()

13

In [273]:
val_data['BsmtFinType1'].unique()

array(['Unf', 'GLQ', 'LwQ', 'Rec', 'ALQ', 'BLQ', nan], dtype=object)

In [274]:
val_data['BsmtFinType1']=val_data['BsmtFinType1'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [275]:
#BsmtFinType2 Column

In [276]:
train_data['BsmtFinType2'].isnull().sum()

24

In [277]:
train_data['BsmtFinType2'].unique()

array(['Unf', 'LwQ', nan, 'Rec', 'BLQ', 'ALQ', 'GLQ'], dtype=object)

In [278]:
train_data['BsmtFinType2']=train_data['BsmtFinType2'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [279]:
val_data['BsmtFinType2'].isnull().sum()

14

In [280]:
val_data['BsmtFinType2'].unique()

array(['Unf', 'GLQ', 'ALQ', 'Rec', 'LwQ', nan, 'BLQ'], dtype=object)

In [281]:
val_data['BsmtFinType2']=val_data['BsmtFinType2'].replace(np.NaN,"No Basement")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [282]:
#Electrical column

In [283]:
train_data['Electrical'].isnull().sum()

0

In [284]:
train_data['Electrical'].unique()

array(['SBrkr', 'FuseA', 'FuseF', 'Mix', 'FuseP'], dtype=object)

In [285]:
train_data['Electrical'].mode().values[0]

'SBrkr'

In [286]:
train_data['Electrical']=train_data['Electrical'].replace(np.NaN,train_data['Electrical'].mode().values[0])

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [287]:
val_data['Electrical'].isnull().sum()

1

In [288]:
val_data['Electrical'].unique()

array(['SBrkr', 'FuseA', 'FuseP', 'FuseF', nan], dtype=object)

In [289]:
val_data['Electrical'].mode().values[0]

'SBrkr'

In [290]:
val_data['Electrical']=val_data['Electrical'].replace(np.NaN,val_data['Electrical'].mode().values[0])

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [291]:
#FireplaceQu Column

In [292]:
train_data['FireplaceQu'].isnull().sum()

417

In [293]:
train_data['FireplaceQu'].unique()

array(['TA', 'Gd', nan, 'Po', 'Ex', 'Fa'], dtype=object)

In [294]:
train_data['FireplaceQu']=train_data['FireplaceQu'].replace(np.NaN,"No Fireplace")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [295]:
val_data['FireplaceQu'].isnull().sum()

273

In [296]:
val_data['FireplaceQu'].unique()

array(['Gd', 'TA', nan, 'Ex', 'Fa', 'Po'], dtype=object)

In [297]:
val_data['FireplaceQu']=val_data['FireplaceQu'].replace(np.NaN,"No Fireplace")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [298]:
# GarageType column

In [299]:
train_data['GarageType'].isnull().sum()

45

In [300]:
train_data['GarageType'].unique()

array(['Attchd', 'Detchd', 'Basment', nan, 'BuiltIn', '2Types', 'CarPort'], dtype=object)

In [301]:
train_data['GarageType']=train_data['GarageType'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [302]:
val_data['GarageType'].isnull().sum()

36

In [303]:
val_data['GarageType'].unique()

array(['Attchd', 'BuiltIn', 'Detchd', nan, '2Types', 'CarPort', 'Basment'], dtype=object)

In [304]:
val_data['GarageType']=val_data['GarageType'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [305]:
# GarageFinish column

In [306]:
train_data['GarageFinish'].isnull().sum()

45

In [307]:
train_data['GarageFinish'].unique()

array(['RFn', 'Unf', 'Fin', nan], dtype=object)

In [308]:
train_data['GarageFinish']=train_data['GarageFinish'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [309]:
val_data['GarageFinish'].isnull().sum()

36

In [310]:
val_data['GarageFinish'].unique()

array(['Fin', 'RFn', 'Unf', nan], dtype=object)

In [311]:
val_data['GarageFinish']=val_data['GarageFinish'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [312]:
#GarageQual Column

In [313]:
train_data['GarageQual'].isnull().sum()

45

In [314]:
train_data['GarageQual'].unique()

array(['TA', nan, 'Fa', 'Gd', 'Ex', 'Po'], dtype=object)

In [315]:
train_data['GarageQual']=train_data['GarageQual'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [316]:
val_data['GarageQual'].isnull().sum()

36

In [317]:
val_data['GarageQual'].unique()

array(['TA', nan, 'Fa', 'Gd'], dtype=object)

In [318]:
val_data['GarageQual']=val_data['GarageQual'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [319]:
#GarageCond Column

In [320]:
train_data['GarageCond'].isnull().sum()

45

In [321]:
train_data['GarageCond'].unique()

array(['TA', nan, 'Fa', 'Gd', 'Po', 'Ex'], dtype=object)

In [322]:
train_data['GarageCond']=train_data['GarageCond'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [323]:
val_data['GarageCond'].isnull().sum()

36

In [324]:
val_data['GarageCond'].unique()

array(['TA', nan, 'Fa', 'Gd'], dtype=object)

In [325]:
val_data['GarageCond']=val_data['GarageCond'].replace(np.NaN,"No Garage")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [326]:
#PoolQC column

In [327]:
train_data['PoolQC'].isnull().sum()

871

In [328]:
train_data['PoolQC'].unique()

array([nan, 'Ex', 'Gd', 'Fa'], dtype=object)

In [329]:
train_data['PoolQC']=train_data['PoolQC'].replace(np.NaN,"No Pool")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [330]:
val_data['PoolQC'].isnull().sum()

582

In [331]:
val_data['PoolQC'].unique()

array([nan, 'Gd', 'Fa'], dtype=object)

In [332]:
val_data['PoolQC']=val_data['PoolQC'].replace(np.NaN,"No Pool")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [333]:
#Fence Column

In [334]:
train_data['Fence'].isnull().sum()

702

In [335]:
train_data['Fence'].unique()

array(['MnPrv', nan, 'GdPrv', 'GdWo', 'MnWw'], dtype=object)

In [336]:
train_data['Fence']=train_data['Fence'].replace(np.NaN,"No Fence")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [337]:
val_data['Fence'].isnull().sum()

477

In [338]:
val_data['Fence'].unique()

array([nan, 'GdPrv', 'MnPrv', 'GdWo', 'MnWw'], dtype=object)

In [339]:
val_data['Fence']=val_data['Fence'].replace(np.NaN,"No Fence")

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [340]:
#MiscFeature Column

In [341]:
train_data['MiscFeature'].isnull().sum()

838

In [342]:
train_data['MiscFeature'].unique()

array(['Shed', nan, 'Othr', 'Gar2', 'TenC'], dtype=object)

In [343]:
train_data['MiscFeature']=train_data['MiscFeature'].replace(np.NaN,'None')

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [344]:
val_data['MiscFeature'].isnull().sum()

568

In [345]:
val_data['MiscFeature'].unique()

array([nan, 'Shed', 'Gar2', 'Othr'], dtype=object)

In [346]:
val_data['MiscFeature']=val_data['MiscFeature'].replace(np.NaN,'None')

C:\Users\Rithvik\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [347]:
train_data.isnull().sum()

s                0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
                ..
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
FireplaceQu      0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
PoolQC      

In [348]:
#From the Above code we can see that all the columns now are not having any null values.

In [349]:
#Now we need to convert the categorical variables to numerical by using average by level.

In [350]:
train_data.columns

Index(['s', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive'

In [351]:
numerical_columns=train_data._get_numeric_data().columns

In [352]:
numerical_columns

Index(['s', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [353]:
categorical_columns=list(set(train_data.columns)-set(numerical_columns))

In [354]:
categorical_columns

['BsmtExposure',
 'HeatingQC',
 'Fence',
 'RoofStyle',
 'BldgType',
 'MasVnrType',
 'Functional',
 'Foundation',
 'MiscFeature',
 'BsmtCond',
 'BsmtFinType2',
 'Exterior2nd',
 'RoofMatl',
 'ExterCond',
 'FireplaceQu',
 'KitchenQual',
 'Street',
 'LandSlope',
 'PavedDrive',
 'BsmtQual',
 'Heating',
 'Condition1',
 'Condition2',
 'HouseStyle',
 'ExterQual',
 'LotConfig',
 'CentralAir',
 'SaleType',
 'LandContour',
 'MSZoning',
 'Alley',
 'GarageQual',
 'GarageType',
 'LotShape',
 'PoolQC',
 'GarageCond',
 'Exterior1st',
 'SaleCondition',
 'Electrical',
 'Neighborhood',
 'GarageFinish',
 'Utilities',
 'BsmtFinType1']

In [355]:
len(categorical_columns)

43

In [356]:
#Average By Level for categorical columns

In [357]:
train_data_copy=train_data.copy()

for each_column in categorical_columns:
    new_col_name=each_column+"_encode"
    #now lets create a dictionary for each of the levels in each categorical column
    dict={}
    
    for level in train_data[each_column].unique():
        dict[level]=train_data[train_data[each_column]==level]['SalePrice'].mean()
        
    train_data_copy[new_col_name]=train_data[each_column].apply(lambda i: dict[i])

In [358]:
train_data_copy

,s,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,LotShape_encode,PoolQC_encode,GarageCond_encode,Exterior1st_encode,SaleCondition_encode,Electrical_encode,Neighborhood_encode,GarageFinish_encode,Utilities_encode,BsmtFinType1_encode
539,540,20,RL,68.0,11423,Pave,No Alley Access,Reg,Lvl,AllPub,...,162897.237347,176474.559127,184945.859873,207522.464286,171991.713267,183510.461538,204203.566265,201957.974576,177294.675429,234410.458167
791,792,80,RL,68.0,11333,Pave,No Alley Access,IR1,Lvl,AllPub,...,200882.727273,176474.559127,184945.859873,158618.902256,171991.713267,183510.461538,157443.939394,201957.974576,177294.675429,160814.122137
816,817,20,RL,68.0,11425,Pave,No Alley Access,IR1,Lvl,AllPub,...,200882.727273,176474.559127,184945.859873,189960.294118,171991.713267,183510.461538,145773.936170,201957.974576,177294.675429,148553.967033
989,990,60,FV,65.0,8125,Pave,No Alley Access,Reg,Lvl,AllPub,...,162897.237347,176474.559127,184945.859873,207522.464286,263357.039474,183510.461538,219119.234043,201957.974576,177294.675429,160366.077220
890,891,50,RL,60.0,8064,Pave,No Alley Access,Reg,Lvl,AllPub,...,162897.237347,176474.559127,184945.859873,150358.977099,171991.713267,183510.461538,145773.936170,139514.384416,177294.675429,160366.077220
1291,1292,160,RM,21.0,1680,Pave,No Alley Access,Reg,Lvl,AllPub,...,162897.237347,176474.559127,184945.859873,240769.518519,171991.713267,183510.461538,105433.333333,139514.384416,177294.675429,160814.122137
272,273,60,RL,92.0,11764,Pave,No Alley Access,IR1,Lvl,AllPub,...,200882.727273,176474.559127,184945.859873,207522.464286,171991.713267,183510.461538,322322.689655,234660.814286,177294.675429,234410.458167
794,795,60,RL,68.0,10832,Pave,No Alley Access,IR1,Lvl,AllPub,...,200882.727273,176474.559127,184945.859873,150358.977099,171991.713267,183510.461538,193959.043478,234660.814286,177294.675429,160366.077220
1019,1020,120,RL,43.0,3013,Pave,No Alley Access,Reg,Lvl,AllPub,...,162897.237347,176474.559127,184945.859873,207522.464286,171991.713267,183510.461538,188760.900000,234660.814286,177294.675429,234410.458167
515,516,20,RL,94.0,12220,Pave,No Alley Access,Reg,Lvl,AllPub,...,162897.237347,176474.559127,184945.859873,240769.518519,263357.039474,183510.461538,308062.930233,234660.814286,177294.675429,234410.458167


In [359]:
#Now removing Categorical columns from the train_data_copy dataset

In [360]:
train_data_copy['Heating'].nunique()

6

In [361]:
train_data_copy['Heating_encode'].nunique()

6

In [362]:
#checking number of columns in new dataset with encoded columns
len(train_data_copy.columns)-len(train_data.columns)

43

In [363]:
train_data_copy=train_data_copy.drop(categorical_columns,axis=1)

In [364]:
len(train_data_copy.columns)

81

In [365]:
#Running XgBoost on the model

In [366]:
x=train_data_copy.drop('SalePrice',axis=1)
x=x.drop('s',axis=1)

In [367]:
x.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'BsmtExposure_encode', 'HeatingQC_encode',
       'Fence_encode', 'RoofStyle_encode', 'BldgType_encode',
       'MasVnrType_encode', 'Functional_encode', 'Foundation_encode',
       'MiscFeature_encode', 'BsmtCond_encode', 'BsmtFinType2_encode',
       'Exterior2nd_encode', 'RoofMatl_encode', 'ExterCond_encode',
       'FireplaceQu_encode', 'KitchenQual_encode', 'Street_encode',
       'LandSlope_encode', 'PavedDrive_encode', 'BsmtQual_enco

In [368]:
len(x.columns)

79

In [369]:
y=train_data['SalePrice']

In [370]:
model=XGBRegressor(max_depth=4,n_estimators=500,learning_rate=0.01,colsample_bylevel=0.4, colsample_bytree=0.4,reg_alpha=0.5)
model.fit(x,y)

XGBRegressor(base_score=0.5, colsample_bylevel=0.4, colsample_bytree=0.4,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0.5, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [371]:
pred_1=model.predict(x)

In [372]:
y_array = np.array(y)
rmse = sqrt(mean_squared_error(y_array, pred_1))
rmse

14344.874547106147

In [373]:
r2_score(y_array,pred_1)

0.96554784163672691

In [374]:
#Now checking the performance of the model on the Validation data

In [375]:
val_data_copy=val_data.copy()

for each_column in categorical_columns:
    new_col_name=each_column+"_encode"
    #now lets create a dictionary for each of the levels in each categorical column
    dict={}
    
    for level in val_data[each_column].unique():
        dict[level]=val_data[val_data[each_column]==level]['SalePrice'].mean()
        
    val_data_copy[new_col_name]=val_data[each_column].apply(lambda i: dict[i])

In [376]:
val_data_copy

,s,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,LotShape_encode,PoolQC_encode,GarageCond_encode,Exterior1st_encode,SaleCondition_encode,Electrical_encode,Neighborhood_encode,GarageFinish_encode,Utilities_encode,BsmtFinType1_encode
1051,1052,20,RL,103.0,11175,Pave,No Alley Access,IR1,Lvl,AllPub,...,212968.688995,186286.314433,192151.543438,222973.550725,286149.673469,191639.123616,190238.358209,248026.591549,186429.118151,186277.976608
131,132,60,RL,70.0,12224,Pave,No Alley Access,IR1,Lvl,AllPub,...,212968.688995,186286.314433,192151.543438,222973.550725,179730.519115,191639.123616,191314.848485,202209.575269,186429.118151,236921.616766
599,600,160,RM,24.0,1950,Pave,No Alley Access,Reg,Lvl,AllPub,...,167778.664773,186286.314433,192151.543438,148043.292135,179730.519115,191639.123616,151000.000000,248026.591549,186429.118151,155850.000000
1451,1452,20,RL,78.0,9262,Pave,No Alley Access,Reg,Lvl,AllPub,...,167778.664773,186286.314433,192151.543438,224480.970588,286149.673469,191639.123616,232924.666667,248026.591549,186429.118151,186277.976608
427,428,20,RL,77.0,8593,Pave,No Alley Access,IR1,Lvl,AllPub,...,212968.688995,186286.314433,192151.543438,148043.292135,179730.519115,191639.123616,145969.857143,146779.990909,186429.118151,146242.607143
1431,1432,120,RL,70.0,4928,Pave,No Alley Access,IR1,Lvl,AllPub,...,212968.688995,186286.314433,192151.543438,174742.820513,179730.519115,191639.123616,148562.500000,202209.575269,186429.118151,155850.000000
1307,1308,20,RL,60.0,8072,Pave,No Alley Access,Reg,Lvl,AllPub,...,167778.664773,186286.314433,192151.543438,222973.550725,179730.519115,191639.123616,190238.358209,146779.990909,186429.118151,162690.168539
1082,1083,20,RL,70.0,8749,Pave,No Alley Access,Reg,Lvl,AllPub,...,167778.664773,186286.314433,192151.543438,222973.550725,179730.519115,191639.123616,190238.358209,202209.575269,186429.118151,186277.976608
930,931,20,RL,73.0,8925,Pave,No Alley Access,IR1,HLS,AllPub,...,212968.688995,186286.314433,192151.543438,222973.550725,179730.519115,191639.123616,254628.500000,248026.591549,186429.118151,236921.616766
857,858,60,RL,65.0,8125,Pave,No Alley Access,Reg,Lvl,AllPub,...,167778.664773,186286.314433,192151.543438,169740.224719,179730.519115,191639.123616,191314.848485,248026.591549,186429.118151,186277.976608


In [377]:
val_data_copy=val_data_copy.drop(categorical_columns,axis=1)

In [378]:
len(val_data_copy.columns)

81

In [379]:
x_val=val_data_copy.drop('SalePrice',axis=1)
x_val=x_val.drop('s',axis=1)
y_val=val_data['SalePrice']

In [380]:
pred_val=model.predict(x_val)
val_y_array=np.array(y_val)
rmse_val=sqrt(mean_squared_error(val_y_array,pred_val))

In [381]:
rmse_val

27217.76502683867

In [382]:
r2_score(val_y_array,pred_val)

0.89036824942208004

In [383]:
train_data_copy.drop('s',axis=1).to_csv("a.csv")

In [384]:
val_data_copy.drop('s',axis=1).to_csv("b.csv")

In [385]:
#train_data_copy[[Neighborhood,Neighborhood_encode]]

In [ ]:
#ELASTIC NET

In [573]:
from sklearn.linear_model import ElasticNet
alpha = 0.5
enet = ElasticNet(alpha=alpha,l1_ratio=0.5,max_iter=3000)
pred_enet=enet.fit(x,y).predict(x_val)
r2_enet=r2_score(y_val,pred_enet)
rmse_enet=sqrt(mean_squared_error(y_val,pred_enet))
print(rmse_enet)
r2_enet

57478.02254750162


C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.51108308269667913

### TEST SET

In [386]:
# Now repeating the whole set of process for the test set inorder to predict the sales proce of houses in the test set

In [387]:
### reading the datset

test_data=pd.read_csv("test.csv")

In [388]:
#First we need to deal with the missing values
#There are 2 columns with missing values LotFrontage and GarageYrBlt
#LotFrontage is a numerical column and hence we impute it with the median 
#We feel the GarageYrBlt is not a useful column in 

In [389]:
test_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [390]:
test_data['MasVnrType'].isnull().sum()

16

In [391]:
test_data['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', nan], dtype=object)

In [392]:
# now we need to impute the missing values in test set
# there are 2 columns with missing values "Lot Frontage" and "GarageYrBlt"

In [393]:
#Imputing the training data set columns

In [394]:
imp=Imputer(missing_values="NaN", strategy="median",axis=1 ) #specify axis
q = imp.fit_transform(test_data["LotFrontage"]).T #perform a transpose operation
test_data['LotFrontage']=q
print (test_data)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0     1461          20       RH         80.0    11622   Pave   NaN      Reg   
1     1462          20       RL         81.0    14267   Pave   NaN      IR1   
2     1463          60       RL         74.0    13830   Pave   NaN      IR1   
3     1464          60       RL         78.0     9978   Pave   NaN      IR1   
4     1465         120       RL         43.0     5005   Pave   NaN      IR1   
5     1466          60       RL         75.0    10000   Pave   NaN      IR1   
6     1467          20       RL         67.0     7980   Pave   NaN      IR1   
7     1468          60       RL         63.0     8402   Pave   NaN      IR1   
8     1469          20       RL         85.0    10176   Pave   NaN      Reg   
9     1470          20       RL         70.0     8400   Pave   NaN      Reg   
10    1471         120       RH         26.0     5858   Pave   NaN      IR1   
11    1472         160       RM         21.0     168

C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [395]:
test_data['LotFrontage'].median()

67.0

In [396]:
imp=Imputer(missing_values="NaN",strategy="most_frequent",axis=1)
q=imp.fit_transform(test_data['GarageYrBlt']).T
test_data['GarageYrBlt']=q
print (test_data['GarageYrBlt'])

0       1961.0
1       1958.0
2       1997.0
3       1998.0
4       1992.0
5       1993.0
6       1992.0
7       1998.0
8       1990.0
9       1970.0
10      1999.0
11      1971.0
12      1997.0
13      1975.0
14      1975.0
15      2009.0
16      2009.0
17      2005.0
18      2005.0
19      2003.0
20      2002.0
21      2006.0
22      2005.0
23      2006.0
24      2004.0
25      2004.0
26      1998.0
27      2005.0
28      2009.0
29      2005.0
         ...  
1429    1925.0
1430    1957.0
1431    2005.0
1432    2005.0
1433    2005.0
1434    2005.0
1435    2004.0
1436    1979.0
1437    1978.0
1438    2001.0
1439    1975.0
1440    1958.0
1441    2000.0
1442    2005.0
1443    2005.0
1444    1951.0
1445    1997.0
1446    1977.0
1447    1968.0
1448    1970.0
1449    2005.0
1450    1972.0
1451    1969.0
1452    1970.0
1453    2005.0
1454    2005.0
1455    1970.0
1456    1960.0
1457    2005.0
1458    1993.0
Name: GarageYrBlt, dtype: float64


C:\Users\Rithvik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [397]:
test_data['GarageYrBlt'].isnull().sum()

0

In [398]:
test_data['LotFrontage'].isnull().sum()

0

In [399]:
#There are many columns in the data which have "None" as a value which actually has a meaning.

In [400]:
a=pd.DataFrame(test_data.isnull().sum())
a['count']=pd.DataFrame(test_data.isnull().sum())

In [401]:
a[a['count']>0]

,0,count
MSZoning,4,4
Alley,1352,1352
Utilities,2,2
Exterior1st,1,1
Exterior2nd,1,1
MasVnrType,16,16
MasVnrArea,15,15
BsmtQual,44,44
BsmtCond,45,45
BsmtExposure,44,44


In [402]:
len(a[a['count']>0])

31

In [403]:
#MSZoning COlumn

In [404]:
test_data['MSZoning'].isnull().sum()

4

In [405]:
test_data['MSZoning']=test_data['MSZoning'].replace(np.NaN,test_data['MSZoning'].mode().values[0])

In [406]:
test_data['MSZoning'].isnull().sum()

0

In [407]:
#Alley Column

In [408]:
test_data['Alley'].isnull().sum()

1352

In [409]:
test_data['Alley']=test_data['Alley'].replace(np.NaN, 'No Alley Access')

In [410]:
#Utilities Column

In [411]:
test_data['Utilities'].isnull().sum()

2

In [412]:
test_data['Utilities']=test_data['Utilities'].replace(np.NaN,test_data['Utilities'].mode().values[0])

In [413]:
#Exterior 1st Column

In [414]:
test_data['Exterior1st'].isnull().sum()

1

In [415]:
test_data['Exterior1st']=test_data['Exterior1st'].replace(np.NaN,test_data['Exterior1st'].mode().values[0])

In [416]:
#Exterior 2nd Column

In [417]:
test_data['Exterior2nd'].isnull().sum()

1

In [418]:
test_data['Exterior2nd']=test_data['Exterior2nd'].replace(np.NaN,test_data['Utilities'].mode().values[0])

In [419]:
#MasVnrType Column

In [420]:
test_data['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', nan], dtype=object)

In [421]:
test_data['MasVnrType']=test_data['MasVnrType'].replace(np.NaN,test_data['MasVnrType'].mode().values[0])

In [422]:
test_data['MasVnrType'].isnull().sum()

0

In [423]:
#MasVnrArea Column

In [424]:
test_data['MasVnrArea'].isnull().sum()

15

In [425]:
test_data['MasVnrArea'].median()

0.0

In [426]:
test_data['MasVnrArea']=test_data['MasVnrArea'].replace(np.NaN,test_data['MasVnrArea'].median())

In [427]:
#BsmtQual Column

In [428]:
test_data['BsmtQual'].isnull().sum()

44

In [429]:
test_data['BsmtQual'].unique()

array(['TA', 'Gd', 'Ex', 'Fa', nan], dtype=object)

In [430]:
test_data['BsmtQual']=test_data['BsmtQual'].replace(np.NaN,"No Basement")

In [431]:
#BsmtCond Column

In [432]:
test_data['BsmtCond'].isnull().sum()

45

In [433]:
test_data['BsmtCond'].unique()

array(['TA', 'Po', 'Fa', 'Gd', nan], dtype=object)

In [434]:
test_data['BsmtCond']=test_data['BsmtCond'].replace(np.NaN,"No Basement")

In [435]:
#BsmtExposure column

In [436]:
test_data['BsmtExposure'].isnull().sum()

44

In [437]:
test_data['BsmtExposure'].unique()

array(['No', 'Gd', 'Mn', 'Av', nan], dtype=object)

In [438]:
test_data['BsmtExposure']=test_data['BsmtExposure'].replace(np.NaN,"No Basement")

In [439]:
#BsmtFinType1 Column

In [440]:
test_data['BsmtFinType1'].isnull().sum()

42

In [441]:
test_data['BsmtFinType1'].unique()

array(['Rec', 'ALQ', 'GLQ', 'Unf', 'BLQ', 'LwQ', nan], dtype=object)

In [442]:
test_data['BsmtFinType1']=test_data['BsmtFinType1'].replace(np.NaN,"No Basement")

In [443]:
#BsmtFinType2 Column

In [444]:
test_data['BsmtFinType2'].isnull().sum()

42

In [445]:
test_data['BsmtFinType2'].unique()

array(['LwQ', 'Unf', 'Rec', 'BLQ', 'GLQ', 'ALQ', nan], dtype=object)

In [446]:
test_data['BsmtFinType2']=test_data['BsmtFinType2'].replace(np.NaN,"No Basement")

In [447]:
#BsmtFinSF1 Column

In [448]:
test_data['BsmtFinSF1'].isnull().sum()

1

In [449]:
test_data['BsmtFinSF1'].median()

350.5

In [450]:
test_data['BsmtFinSF1']=test_data['BsmtFinSF1'].replace(np.NaN,test_data['BsmtFinSF1'].median())

In [451]:
#BsmtFinSF2 Column

In [452]:
test_data['BsmtFinSF2'].isnull().sum()

1

In [453]:
test_data['BsmtFinSF2'].median()

0.0

In [454]:
test_data['BsmtFinSF2']=test_data['BsmtFinSF2'].replace(np.NaN,test_data['BsmtFinSF2'].median())

In [455]:
#BsmtUnfSF Column

In [456]:
test_data['BsmtUnfSF'].isnull().sum()

1

In [457]:
test_data['BsmtUnfSF'].median()

460.0

In [458]:
test_data['BsmtUnfSF']=test_data['BsmtUnfSF'].replace(np.NaN,test_data['BsmtUnfSF'].median())

In [459]:
#TotalBsmtSF Column

In [460]:
test_data['TotalBsmtSF'].isnull().sum()

1

In [461]:
test_data['TotalBsmtSF'].median()

988.0

In [462]:
test_data['TotalBsmtSF']=test_data['TotalBsmtSF'].replace(np.NaN,test_data['TotalBsmtSF'].median())

In [463]:
#BsmtFullBath column

In [464]:
test_data['BsmtFullBath'].isnull().sum()

2

In [465]:
test_data['BsmtFullBath'].median()

0.0

In [466]:
test_data['BsmtFullBath']=test_data['BsmtFullBath'].replace(np.NaN,test_data['BsmtFullBath'].median())

In [467]:
#BsmtHalfBath column

In [468]:
test_data['BsmtHalfBath'].isnull().sum()

2

In [469]:
test_data['BsmtHalfBath'].median()

0.0

In [470]:
test_data['BsmtHalfBath']=test_data['BsmtHalfBath'].replace(np.NaN,test_data['BsmtHalfBath'].median())

In [471]:
#KitchenQual Column

In [472]:
test_data['KitchenQual'].isnull().sum()

1

In [473]:
test_data['KitchenQual'].mode().values[0]

'TA'

In [474]:
test_data['KitchenQual']=test_data['KitchenQual'].replace(np.NaN,test_data['KitchenQual'].mode().values[0])

In [475]:
#Functional column

In [476]:
test_data['Functional'].isnull().sum()

2

In [477]:
test_data['Functional'].mode().values[0]

'Typ'

In [478]:
test_data['Functional']=test_data['Functional'].replace(np.NaN,test_data['Functional'].mode().values[0])

In [479]:
#FireplaceQu Column

In [480]:
test_data['FireplaceQu'].isnull().sum()

730

In [481]:
test_data['FireplaceQu'].unique()

array([nan, 'TA', 'Gd', 'Po', 'Fa', 'Ex'], dtype=object)

In [482]:
test_data['FireplaceQu']=test_data['FireplaceQu'].replace(np.NaN,"No Fireplace")

In [483]:
# GarageType column

In [484]:
test_data['GarageType'].isnull().sum()

76

In [485]:
test_data['GarageType'].unique()

array(['Attchd', 'Detchd', 'BuiltIn', nan, 'Basment', '2Types', 'CarPort'], dtype=object)

In [486]:
test_data['GarageType']=test_data['GarageType'].replace(np.NaN,"No Garage")

In [487]:
# GarageFinish column

In [488]:
test_data['GarageFinish'].isnull().sum()

78

In [489]:
test_data['GarageFinish'].unique()

array(['Unf', 'Fin', 'RFn', nan], dtype=object)

In [490]:
test_data['GarageFinish']=test_data['GarageFinish'].replace(np.NaN,"No Garage")

In [491]:
#Garaage Cars COlumn

In [492]:
test_data['GarageCars'].isnull().sum()

1

In [493]:
test_data['GarageCars'].median()

2.0

In [494]:
test_data['GarageCars']=test_data['GarageCars'].replace(np.NaN,test_data['GarageCars'].median())

In [495]:
#Garaage area COlumn

In [496]:
test_data['GarageArea'].isnull().sum()

1

In [497]:
test_data['GarageArea'].median()

480.0

In [498]:
test_data['GarageArea']=test_data['GarageArea'].replace(np.NaN,test_data['GarageArea'].median())

In [499]:
#GarageQual Column

In [500]:
test_data['GarageQual'].isnull().sum()

78

In [501]:
test_data['GarageQual'].unique()

array(['TA', nan, 'Fa', 'Gd', 'Po'], dtype=object)

In [502]:
test_data['GarageQual']=test_data['GarageQual'].replace(np.NaN,"No Garage")

In [503]:
#GarageCond Column

In [504]:
test_data['GarageCond'].isnull().sum()

78

In [505]:
test_data['GarageCond'].unique()

array(['TA', nan, 'Fa', 'Gd', 'Po', 'Ex'], dtype=object)

In [506]:
test_data['GarageCond']=test_data['GarageCond'].replace(np.NaN,"No Garage")

In [507]:
#PoolQC column

In [508]:
test_data['PoolQC'].isnull().sum()

1456

In [509]:
test_data['PoolQC'].unique()

array([nan, 'Ex', 'Gd'], dtype=object)

In [510]:
test_data['PoolQC']=test_data['PoolQC'].replace(np.NaN,"No Pool")

In [511]:
#Fence Column

In [512]:
test_data['Fence'].isnull().sum()

1169

In [513]:
test_data['Fence'].unique()

array(['MnPrv', nan, 'GdPrv', 'GdWo', 'MnWw'], dtype=object)

In [514]:
test_data['Fence']=test_data['Fence'].replace(np.NaN,"No Fence")

In [515]:
#MiscFeature Column

In [516]:
test_data['MiscFeature'].isnull().sum()

1408

In [517]:
test_data['MiscFeature'].unique()

array([nan, 'Gar2', 'Shed', 'Othr'], dtype=object)

In [518]:
test_data['MiscFeature']=test_data['MiscFeature'].replace(np.NaN,'None')

In [519]:
#Sale Type Column

In [520]:
test_data['SaleType'].isnull().sum()

1

In [521]:
test_data['SaleType'].mode().values[0]

'WD'

In [522]:
test_data['SaleType']=test_data['SaleType'].replace(np.NaN,test_data['SaleType'].mode().values[0])

In [523]:
#Testing if there are anymore nulls

In [524]:
test_data.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
                ..
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
FireplaceQu      0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea    

In [525]:
#From the Above code we can see that all the columns now are not having any null values.

In [526]:
#Now we need to convert the categorical variables to numerical by using average by level.

In [527]:
test_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [528]:
numerical_columns=test_data._get_numeric_data().columns

In [529]:
numerical_columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [530]:
categorical_columns=list(set(test_data.columns)-set(numerical_columns))

In [531]:
categorical_columns

['BsmtExposure',
 'HeatingQC',
 'Fence',
 'RoofStyle',
 'BldgType',
 'MasVnrType',
 'Functional',
 'Foundation',
 'MiscFeature',
 'BsmtCond',
 'BsmtFinType2',
 'Exterior2nd',
 'RoofMatl',
 'ExterCond',
 'FireplaceQu',
 'KitchenQual',
 'Street',
 'LandSlope',
 'PavedDrive',
 'BsmtQual',
 'Heating',
 'Condition1',
 'Condition2',
 'HouseStyle',
 'ExterQual',
 'LotConfig',
 'CentralAir',
 'SaleType',
 'LandContour',
 'MSZoning',
 'Alley',
 'GarageQual',
 'GarageType',
 'LotShape',
 'PoolQC',
 'GarageCond',
 'Exterior1st',
 'SaleCondition',
 'Electrical',
 'Neighborhood',
 'GarageFinish',
 'Utilities',
 'BsmtFinType1']

In [532]:
len(categorical_columns)

43

In [533]:
#Average By Level for categorical columns

In [534]:
test_data_copy=test_data.copy()

for each_column in categorical_columns:
    new_col_name=each_column+"_encode"
    #now lets create a dictionary for each of the levels in each categorical column
    dict={}
    
    for level in test_data[each_column].unique():
        dict[level]=test_data[test_data[each_column]==level]['SalePrice'].mean()
        
    test_data_copy[new_col_name]=test_data[each_column].apply(lambda i: dict[i])

KeyError: 'SalePrice'

In [ ]:
train_data_copy

In [ ]:
#Now removing Categorical columns from the train_data_copy dataset

In [ ]:
train_data_copy['Heating'].nunique()

In [ ]:
train_data_copy['Heating_encode'].nunique()

In [ ]:
#checking number of columns in new dataset with encoded columns
len(train_data_copy.columns)-len(train_data.columns)

In [ ]:
train_data_copy=train_data_copy.drop(categorical_columns,axis=1)

In [ ]:
len(train_data_copy.columns)

In [ ]:
#Running XgBoost on the model

In [ ]:
x=train_data_copy.drop('SalePrice',axis=1)
x=x.drop('s',axis=1)

In [ ]:
x.columns

In [ ]:
len(x.columns)

In [ ]:
y=train_data['SalePrice']

In [ ]:
model=XGBRegressor(max_depth=4,n_estimators=500,learning_rate=0.01,colsample_bylevel=0.4, colsample_bytree=0.4,reg_alpha=0.5)
model.fit(x,y)

In [ ]:
pred_1=model.predict(x)

In [ ]:
y_array = np.array(y)
rmse = sqrt(mean_squared_error(y_array, pred_1))
rmse

In [ ]:
r2_score(y_array,pred_1)